In [4]:
import pandas as pd

# Function to extract and combine food truck data with a sheet range
def extract_and_combine_foodtruck_data(file_path, sheet_start=None, sheet_end=None):
    excel_file = pd.ExcelFile(file_path)
    combined_data = {}

    # Determine the sheets to process based on the range
    sheets_to_process = excel_file.sheet_names[sheet_start:sheet_end]

    for sheet in sheets_to_process:
        # Read the current sheet
        df = pd.read_excel(file_path, sheet_name=sheet)
        
        # Extract the second column where times and food trucks are stored
        second_column = df.iloc[:, 1]
        current_time = None
        
        for i, value in enumerate(second_column):
            if isinstance(value, str) and "-" in value and ":" in value:
                # Detect a time interval
                current_time = value.strip()
                if current_time not in combined_data:
                    combined_data[current_time] = []
            elif value == "Name/Meal Period":
                # Skip this row
                continue
            elif current_time and value == "Total:":
                # Stop collecting for the current time interval
                current_time = None
            elif current_time and pd.notna(value):
                # Add food truck names under the current time interval
                combined_data[current_time].append(value)
    
    # Convert the combined_data dictionary into a DataFrame
    combined_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in combined_data.items()]))
    return combined_df

# Apply the function to your Excel file
#file_path = "UCLA Ticket Count Sheet - 6.12.23.xlsx"  # Replace with the path to your Excel file
#sheet_start = 67  # Starting index (0-based, so 4 corresponds to sheet 5)
#sheet_end = 380   # Ending index (non-inclusive, so 10 corresponds to sheet 10)
#combined_foodtruck_data = extract_and_combine_foodtruck_data(file_path, sheet_start, sheet_end)

# Save the result to a CSV file
#csv_combined_foodtruck.to_csv("combined_foodtruck_schedule.csv", index=False)

#print(combined_foodtruck_data)

In [9]:
import numpy as np

# Load dataset
combined_foodtruck_data = pd.read_csv("combined_foodtruck_schedule.csv")

# get rid of entries with asterisk b/c * indicates cancelled/did not show
combined_df = combined_foodtruck_data.applymap(lambda x: np.nan if '*' in str(x) else x)

def rename_food_trucks(df):
    replacements = [
        (r'8E8', '8E8 Thai Street Food'),
        (r'Belly', "Belly's Sliders"),
        (r'Smile', 'Smile Hot Dog'),
        (r'Aloha', 'Aloha Fridays'),
        (r'Bittie|bittie', 'Bittie Bitez'),
        (r'Bollywwood Kitchen|Bolywood Kitchen|Bollywood', 'Bollywood Kitchen'),
        (r'Cerda', 'Cerda Vega'),
        (r'Deli ', 'Deli Doctor '),
        (r'grain|Grain', 'Flaming Grain'),
        (r'Habibi', 'Habibi Shack'),
        (r'Heritage', 'Heritage'),
        (r'Aus|Autyn', "Austyn's Burgers"),
        (r'Kogi', 'Kogi'),
        (r'Yuna', "Yuna's Bob"),
        (r'Wafl', 'Wafl'),
        (r'Venice', 'Venice Gelato'),
        (r'Uncle|Unle', "Uncle Al's"),
        (r'TrapiYaki', 'Trapiyaki'),
        (r'Tokyo', 'Tokyo Style'),
        (r'Cocina', 'Thai Mex Cocina'),
        (r'Taste', 'Taste Collective Burger'),
        (r'Cartel', 'Cartel Tacos'),
        (r'Sweet', 'Sweets on Wheels'),
        (r'Sugo', 'Sugo Italiano'),
        (r'Bye', 'StopBye Café'),
        (r'Stout', 'Stout Burgers'),
        (r'europa|Europa', 'Dulce Europa'),
        (r'Savage', 'Savage Tacos'),
        (r'Poutine', 'Poutine Brothers'),
        (r'Pinch', 'Pinch of Flavor'),
        (r'Philly|Phily', "Philly Jay's"),
        (r'Pacifico', 'Pacifico Charbroiled'),
        (r'Original', 'Original Herbivore'),
        (r'Paradise', 'Paradise'),
        (r'ML E', 'ML Eats'),
        (r'Manna', 'Manna from Heaven'),
        (r'Perro', 'Perro'),
        (r'Kal', 'Kalamaki Greek'),
        (r'Fusion|fusion', 'Go Fusion'),
        (r'Cream', 'Creamy Boys'),
        (r'Dina', "Dina's Dumplings"),
        (r'Bison', 'Bison Burger'),
        (r'Bunz', 'Bunz Gourmet'),
        (r'Chicken|chicken', 'Flaming Hot Chicken'),
        (r'Flaming Hot|Flamng Hot', 'Flaming Hot Chicken'),
        (r'\*Streets of Vietnam', 'Streets of Vietnam'),
        (r'Baby|Bbay', "Baby's Burgers"),
        (r'Café Vietname', 'Café Vietnam')
    ]
    
    def replace_if_contains(text):
        for pattern, replacement in replacements:
            if pd.notna(text) and pd.Series(text).str.contains(pattern, regex=True).any():
                return replacement  # Replace the entire string
        return text  # Keep original if no match

    return df.applymap(replace_if_contains)

cleaned_df = rename_food_trucks(combined_df)

# Reshape the data: Count occurrences of each vendor for each time slot
formatted_data = {}

# Iterate through each time slot column and count occurrences of each vendor
for col in cleaned_df.columns:
    vendor_counts = cleaned_df[col].value_counts()
    formatted_data[col] = vendor_counts

# Convert dictionary to a DataFrame and fill missing values with 0
counts_df = pd.DataFrame(formatted_data).fillna(0).astype(int)

# Save the cleaned data
counts_df.to_csv("counts.csv", index=True)

In [8]:
# Load dataset
#counts_df = pd.read_csv("counts.csv")

# Rename the incorrect column name
counts_df.rename(columns={"9:00pm - 12pm": "9:00pm - 12am"}, inplace=True)

def calculate_duration(time_range):
    start, end = time_range.split(" - ")
    
    # Convert to 24-hour format and extract hours
    def to_hours(time_str):
        time, period = time_str[:-2], time_str[-2:]
        hours, minutes = map(int, time.split(":")) if ":" in time else (int(time), 0)
        if period == "pm" and hours != 12:
            hours += 12
        if period == "am" and hours == 12:
            hours = 0
        return hours + minutes / 60
    
    start_hours = to_hours(start)
    end_hours = to_hours(end)

    # Handle the midnight crossover (e.g., 9:00pm - 12am should be 3 hours)
    if end_hours < start_hours:
        end_hours += 24  # Adjust end time to correctly calculate the difference
    
    return end_hours - start_hours

counts_df.reset_index(inplace=True) # resetting index

# Extract time interval columns (excluding the first column with vendor names)
time_columns = counts_df.columns[1:]

# Compute durations for each time interval
durations = {col: calculate_duration(col) for col in time_columns}

# Multiply each column by its respective duration
for col in time_columns:
    counts_df[col] = counts_df[col] * durations[col]

# Add a new column with the sum of each row
counts_df["Total Sum"] = counts_df.iloc[:, 1:].sum(axis=1)
print(counts_df)

                                    index  11:00am - 3pm  5:00pm - 9pm  \
0                    8E8 Thai Street Food          124.0         140.0   
1                           Aloha Fridays           88.0         100.0   
2                             Asian (TBD)            0.0           0.0   
3                        Austyn's Burgers           28.0          48.0   
4                          Baby's Burgers           32.0          28.0   
5                         Belly's Sliders           44.0          28.0   
6                            Bison Burger           72.0          68.0   
7                            Bittie Bitez            8.0           4.0   
8                       Bollywood Kitchen           72.0          92.0   
9                            Bunz Gourmet            8.0          12.0   
10                           Café Vietnam           32.0          48.0   
11                           Cartel Tacos          144.0          92.0   
12                             Cerda V